Ce script vise à utiliser l'API Légifrance, disponible via PISTE.

# 1/ Mise en place
Il nous faut dans un premier temps obtenir notre token : celui-ci s'obtient grâce aux identifiants générés sur le portail PISTE lorsqu'on a accepté les CGU de l'API Légifrance.

In [1]:
import requests
import pandas as pd

# Prompt the user to input client_id and client_secret
# (J'imagine que hardcoder ses identifiants est une mauvaise pratique)
client_id = input("Enter your client_id: ")  # client_id generated by PISTE
client_secret = input("Enter your client_secret: ")  # client_secret generated by PISTE

url = "https://sandbox-oauth.piste.gouv.fr/api/oauth/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url, data=payload, headers=headers)

# Store the access token in a variable
access_token = None
if response.status_code == 200:
    access_token = response.json().get("access_token")
else:
    print("Error:", response.status_code, response.text)


Enter your client_id: bbd26e62-8737-4fe4-ac36-1b7349d14c10
Enter your client_secret: de2af0dc-a78c-44b8-9aa3-81e6d05b3642


# 2/ Accès à la documentation Swagger

In [2]:
import json
import requests

# Charger la documentation Swagger depuis une URL
url = "https://github.com/user-attachments/files/17714249/Legifrance.json"
response = requests.get(url)
swagger_doc = response.json()

# Afficher des informations générales
print("Titre:", swagger_doc['info']['title'])
print("Version:", swagger_doc['info']['version'])
print("Description:", swagger_doc['info'].get('description', 'Pas de description disponible'))

# Parcourir les chemins (endpoints)
print("\nEndpoints disponibles :")
for path, methods in swagger_doc['paths'].items():
    print(f"\nPath : {path}")
    for method, details in methods.items():
        print(f"  Méthode : {method.upper()}")
        print("    Description :", details.get('description', 'Pas de description disponible'))

        # Afficher les paramètres de chaque méthode
        if 'parameters' in details:
            print("    Paramètres :")
            for param in details['parameters']:
                param_type = param.get('type', 'inconnu')  # Définit le type à "inconnu" si la clé "type" n'existe pas
                print(f"      - {param['name']} (type: {param_type}) - {'Obligatoire' if param.get('required') else 'Optionnel'}")

        # Afficher les réponses possibles
        print("    Réponses :")
        for status_code, response in details['responses'].items():
            print(f"      - Code {status_code}: {response.get('description', 'Pas de description')}")


Titre: Légifrance
Version: 2.4.2
Description: 
Afin de faciliter la réutilisation des données juridiques, la DILA met à disposition du public une API de Légifrance.
Ce document a pour objectif de décrire les méthodes de l'API Légifrance, présentées à l'aide du framework Swagger.
Vous pouvez également télécharger une [documentation complémentaire](https://www.legifrance.gouv.fr/contenu/Media/Files/pied-de-page/description-des-tris-et-filtres-de-l-api.xlsx) ainsi que des [exemples d’utilisation](https://www.legifrance.gouv.fr/contenu/Media/Files/pied-de-page/exemples-d-utilisation-de-l-api.docx) de l'API.

Cet accès vous permet d’expérimenter l'API sur un environnement de test et de vous familiariser avec les différentes méthodes.

À noter :
- L'environnement de test peut être indisponible.
- Les appels sont limités par quotas.

Les mises à jour correctives et évolutives sont réalisées dans un premier temps sur cet environnement.
L'API mise à disposition correspond à celle utilisée par l

# 3/ Requêtes pour les prénoms au JORF
Notre objectif, exploratoire dans un premier temps, est d'avoir pour l'année 2024 l'ensemble des nominations. Il nous faudra sans doute récupérer le NOR, qui nous renseignera le ministère et l'année.

Pour l'instant, on voit en regardant l'output qu'il n'y a pas de tag préexistant nous facilitant beaucoup la tâche : il serait chouette que certains codes soient stables, mais j'en doute.

In [3]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Afin de comprendre le format d'un output de nomination, utilisons le NOR d'une nomination donnée
payload = {
    "nor": "ECOP2427670A"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    print(data)  # Affiche le contenu brut de la réponse pour analyse
else:
    print(f"Erreur {response.status_code}: {response.text}")

{'executionTime': 1, 'dereferenced': False, 'id': 'JORFTEXT000050397779_01-01-2999', 'idConteneur': None, 'cid': 'JORFTEXT000050397779', 'title': "Arrêté du 18 octobre 2024 portant nominations au cabinet du ministre de l'économie, des finances et de l'industrie", 'nor': 'ECOP2427670A', 'eli': None, 'alias': None, 'jorfText': 'JORF n°0255 du 26 octobre 2024', 'jurisState': 'Sans état juridique', 'visa': "<p align='left'><br/>Le ministre de l'économie, des finances et de l'industrie,<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000034938597&categorieLien=cid' title='Décret n°2017-1098 du 14 juin 2017'>décret n° 2017-1098 du 14 juin 2017</a> relatif aux collaborateurs du Président de la République et des membres du Gouvernement ;<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000050251629&categorieLien=cid' title='Décret n°2024-892 du 23 septembre 2024'>décret n° 2024-892 du 23 septembre 2024</a> relatif à la composition des cabinets ministériels ;<br/>Vu le <a href='/affichTe

Puis, avec le module search: le code ci-dessous prend les nominations auprès de Barnier lui même ou de ses ministres délégués si je ne dis pas de bêtise.

In [4]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/search/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}


payload = {
    "recherche": {
        "filtres": [
            {
                "valeurs": ["ARRETE"],  # Filtrer uniquement par arrêté, ce qui inclut les nominations
                "facette": "NATURE"
            },
            {
                "dates": {
                    "start": "2024-01-01",
                    "end": "2024-12-31"
                },
                "facette": "DATE_SIGNATURE"
            }
        ],
        "sort": "SIGNATURE_DATE_DESC",
        "fromAdvancedRecherche": False,
        "secondSort": "ID",
        "champs": [
            {
                "criteres": [
                    {
                        "valeur": "cabinet ministre",  # Critère pour filtrer les nominations dans les cabinets ministériels
                        "operateur": "ET",
                        "typeRecherche": "TOUS_LES_MOTS_DANS_UN_CHAMP"
                    }
                ],
                "operateur": "ET",
                "typeChamp": "TITLE"
            }
        ],
        "pageSize": 10,  # Ajustez selon vos besoins
        "operateur": "ET",
        "typePagination": "DEFAUT",
        "pageNumber": 1
    },
    "fond": "LODA_DATE"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    # Extraction des NOR des arrêtés
    nors = [item['nor'] for item in data.get('results', [])]
    print("NOR des nominations en 2024:", nors)
else:
    print("Erreur lors de la requête:", response.status_code, response.text)



NOR des nominations en 2024: ['PRMX2432440A', 'PRMX2432236A', 'PRMX2432079A', 'PRMX2430415A', 'MOMC2426137A', 'MOMC2429204A', 'EAEC2428687A', 'PRMX2429339A', 'MOMC2429076A', 'PRMX2429154A']


Désormais, pour l'ensemble de ces extraits du JO, identifiés par les NOR, nous allons essayer de récuperer les informations souhaitées.

In [20]:
## Celle-ci je la fais à la main et je fais en sorte de comprendre!  (BALISE - MAIN)

import requests
import pandas as pd
import re
import time

# API details
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",  # il faut faire tourner le premier chunk pour avoir l'access token
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# List of NORs to process

# Regex to extract nominee details
regex = r"(M\.|Mme) ([\w\-]+) ([\w\-]+(?: [\w\-]+)?)(?:,| est nommé(e)?) (.*?)(?:,|:|;|\s)?(?: à compter du ([\d]{1,2} [^\d]+ [\d]{4}|\d{1,2}/\d{1,2}/\d{4}))(?:,|:|;|\n)?"
#r"(M\.|Mme) ([\w\-]+) ([\w\-]+(?: [\w\-]+)?)(?:,| est nommée?| est nommée?) (.*?)(?:,|:|;|\s)?(?: à compter du ([\d]{1,2} [^\d]+ [\d]{4}|\d{1,2}/\d{1,2}/\d{4}))(?:,|:|;|\n)?"

# Data collection
all_nominees = []

# Loop through each NOR
for nor in nors:
    payload = {"nor": nor}
    try:
        # Send the request
        response = requests.post(url, headers=headers, json=payload)

        if response.status_code == 200:
            data = response.json()

            # Extract details for each article
            articles = data.get("articles", [])
            for article in articles:
                content = article.get("content", "")

                # Apply regex to find matches
                matches = re.findall(regex, content, re.IGNORECASE | re.DOTALL)

                for match in matches:
                    genre, prenom, nom, _, titre, date = match
                    all_nominees.append({
                        "Genre": genre,
                        "Prénom": prenom,
                        "Nom": nom,
                        "Titre": titre.strip(),
                        "Date de début": date.strip() if date else "Non précisée",
                        "NOR": nor,
                        "Ministre": re.search(r"Fait le .*?<br/>(.*?)</p>", data.get("signers", ""), re.DOTALL).group(1).strip() if "signers" in data else "Non spécifié",
                        "Ministère": re.search(r"cabinet du (.*)|cabinet de la (.*)", data.get("title", ""), re.IGNORECASE).group(1) or re.search(r"cabinet du (.*)|cabinet de la (.*)", data.get("title", ""), re.IGNORECASE).group(2) if re.search(r"cabinet du (.*)|cabinet de la (.*)", data.get("title", ""), re.IGNORECASE) else "Non spécifié"
                    })
        elif response.status_code == 429:
            # Handle rate-limiting
            retry_after = int(response.headers.get("Retry-After", 5))
            print(f"Rate limit hit. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            continue
        else:
            print(f"Error {response.status_code} for NOR {nor}: {response.text}")
    except Exception as e:
        print(f"Exception occurred for NOR {nor}: {e}")

    # Avoid hitting the API rate limit
    time.sleep(1)  # Adjust sleep time based on your successful 429 handling

# Convert to DataFrame
df = pd.DataFrame(all_nominees)

# Display the resulting DataFrame
print(df)

   Genre        Prénom             Nom  \
0    Mme        Sabine         HAMMOND   
1    Mme    Marie-Anne        LAVERGNE   
2    Mme     Madeleine           ROUOT   
3    Mme         Anais  KEITA-GORISSEN   
4    Mme         Sarah           ALLIX   
5     M.        Déobal           GOBIN   
6    Mme      Blandine    DENBY WILKES   
7    Mme      Juliette           MOLLO   
8    Mme        Céline           Bentz   
9     M.  Jean-Jacques           BOREL   
10   Mme        Esther          GARCIA   

                                                Titre     Date de début  \
0   conseillère technique en charge des discours (...   2 décembre 2024   
1   conseillère services financiers et économie in...  24 novembre 2024   
2   conseillère technique en charge de l'argumentaire  12 novembre 2024   
3                        conseillère technique presse  28 novembre 2024   
4   conseillère territoires et services publics au...  18 novembre 2024   
5   conseiller chargé des relations avec les 

In [10]:
print(access_token)
print(nors)

RzKrHFp68ykR8bvixIluc58wmF4fsDdk97B2jgjbp9A8ctA5ZIELXe
['PRMX2432440A', 'PRMX2432236A', 'PRMX2432079A', 'PRMX2430415A', 'MOMC2426137A', 'MOMC2429204A', 'EAEC2428687A', 'PRMX2429339A', 'MOMC2429076A', 'PRMX2429154A']
